# Eigenvalue distribution of the Preconditioned Hessian matrix

In [ ]:
from pipeline_non_linear import NonLinearPipeline

## Parameters

In [ ]:
parameters_dict = {
    'nside': 8,
    'nside_beta': 4,
    'npointings': 500,
    'Nsub': 4,
    'dust_level': 1, #>0
    'dust_model': 'd1', #d0, d1 or d6
    'dust_reconstruction': True, #bool
    'synchrotron_level': 0, #>0
    'synchrotron_model': 's1', #s0 or s1
    'synchrotron_reconstruction': False, #bool
    'frequencies_planck': [100e9, 143e9, 217e9, 353e9],
    'noise_qubic': 0, #0.01,
    'noise_planck': 0, #0.01,
    'planck_coverage_level': 0.2,
    'max_iteration': 5,
    'pcg_tolerance': 1e-16,
    'sigma0': 1e-3,
    'initial_guess': None,
    'verbose': True
}

## Initialization and definition of the gradient

In [ ]:
self = NonLinearPipeline(parameters_dict)

In [ ]:
grad_chi2 = self.get_grad_chi_squared_operator()

In [ ]:
real_sky_patch = {}
real_sky_patch['cmb'] = self.real_sky['cmb'][self.seenpix_qubic, :].copy()
real_sky_patch['dust'] = self.real_sky['dust'][self.seenpix_qubic, :].copy()
real_sky_patch['beta_dust'] = self.real_sky['beta_dust'][self.seenpix_qubic_beta].copy()
real_comp = self.component_combiner(real_sky_patch) # minimum point of the chi^2

## Computation of the Hessian matrix at the minimum point

In [ ]:
nparameters = 6*self.npixel_patch + self.nbeta_patch
sol = grad_chi2(real_comp)
e = np.zeros(nparameters)
epsilon = 1e-5
hessian = np.empty((nparameters, nparameters))
for i in range(nparameters):
    if i % 20 == 0:
        print(i)
    e[i] = epsilon
    hessian[i, :] = (grad_chi2(real_comp + e) - sol) / epsilon
    e[i] = 0

## Plot of the Hessian matrix

In [ ]:
plt.matshow(hessian, norm='log')

## Computation of  the eigenvalues of the Hessian matrix and the preconditioned one

We symmetrize the matrices before the computation to help for numerical stability.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh((hessian.T+hessian)/2)
eigenvalues_preconditioned, _ = np.linalg.eigh(np.diag(np.sqrt(self.HessianInverseDiagonal(real_comp))) @ ((hessian.T+hessian)/2) @ np.diag(np.sqrt(self.HessianInverseDiagonal(real_comp))))


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(eigenvalues[::-1])
ax1.set_yscale('log')
ax1.set_title('Eigenvalues of the Hessian matrix')
ax1.set_xlabel('Eigenvalues numerotation')
ax1.set_ylabel('Eigenvalues')
ax2.plot(np.sort(eigenvalues_preconditioned)[::-1])
ax2.set_yscale('log')
ax2.set_title('Eigenvalues of the preconditioned Hessian matrix')
ax2.set_xlabel('Eigenvalues numerotation')
ax2.set_ylabel('Eigenvalues')
plt.show()
#plt.savefig('eigenvalues.pdf')
#plt.close()

## Plot of the diagonal of the Hessian matrix and the approximated diagonal for preconditioning

In [ ]:
plt.plot(np.diag(hessian))
plt.plot(1/self.HessianInverseDiagonal(real_comp))
plt.yscale('log')

## Plot of the ratio of both

In [ ]:
plt.plot(np.diag(hessian) * self.HessianInverseDiagonal(real_comp))
plt.yscale('log')

## Computation of the condition number of the Hessian matrix before and after preconditioning

In [ ]:
print(np.max(eigenvalues)/np.min(eigenvalues))
print(np.max(eigenvalues_preconditioned)/np.min(eigenvalues_preconditioned))